In [9]:
import torch
from torchvision import datasets, models, transforms

In [10]:
import torch.nn.functional as F
import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt
import cv2


In [11]:
model = models.vgg16(pretrained=True)

In [12]:
last_layer = nn.Linear(4096, 10)
model.classifier[6] = last_layer

In [13]:
if torch.cuda.is_available() == True:
    device = torch.device('cuda:7')
    print(torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
device

device(type='cpu')

In [14]:
batch_size = 128
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def load_data():    
    transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor()])
    transform_test = transforms.Compose([transforms.ToTensor()])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    dataset = {'train': trainset, 'test': testset}
    data_loader = {'train': train_loader, 'test': test_loader}
    return dataset, data_loader

def load_iter(data_loader, data_type):
    bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
    
    if data_type == 'train':
        train_loader = data_loader['train']
        train_iter = tqdm(enumerate(train_loader), total=len(train_loader), unit_scale=batch_size, bar_format=bar_format)
        return train_iter
    elif data_type == 'test':
        test_loader = data_loader['test']
        test_iter = tqdm(enumerate(test_loader), total=len(test_loader), unit_scale=batch_size, bar_format=bar_format)
        return test_iter
    else:
        print('Data Error!!!')

In [15]:
def train(model):
    model.train()
    train_loss = 0
    total = 0
    correct = 0
    step = 0
    train_iter = load_iter(data_loader, 'train')
    batch,label = next(iter(tran_iter))
    
#     for i, (batch, label) in train_iter:

    batch, label = batch.to(device), label.to(device)
    output = model(batch)

    optimizer.zero_grad()
    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = output.max(1)
    total += label.size(0)
    correct += predicted.eq(label).sum().item()

    acc = 100. * correct / total
    train_iter.set_description(f'[{acc:.2f}% ({correct}/{total})]', True)
    print("training complete; accuracy: ", acc)

In [16]:
dataset, data_loader = load_data()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
loss_function = nn.CrossEntropyLoss()

NameError: name 'torchvision' is not defined

In [ ]:
train(model)

In [13]:
def save_model(epoch, acc, optimizer):
    global best_acc
    if acc > best_acc:
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'optimizer': optimizer.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/model_{epoch}.pth')
        best_acc = acc
        print('Saving Model...')

In [ ]:
save_model()

In [20]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0.
    test_iter = load_iter(data_loader, 'test')

    for i, (batch, label) in test_iter:
        print('batch sample:', batch[0].size(), batch[0] )
        
        batch, label = batch.to(device), label.to(device)
        batch = 255. * batch
        
        output = model(batch)
        print("output",output[0])
        loss = loss_function(output, label)
        
        test_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        print('pred', predicted)
        acc = 100. * correct / total
        test_iter.set_description(f'[{acc:.2f}%({correct}/{total})]', True)
    return acc

In [21]:
test(model)

                               0/10112 [00:00<? ?it/s] 

batch sample: torch.Size([3, 32, 32]) tensor([[[0.6196, 0.6235, 0.6471,  ..., 0.5373, 0.4941, 0.4549],
         [0.5961, 0.5922, 0.6235,  ..., 0.5333, 0.4902, 0.4667],
         [0.5922, 0.5922, 0.6196,  ..., 0.5451, 0.5098, 0.4706],
         ...,
         [0.2667, 0.1647, 0.1216,  ..., 0.1490, 0.0510, 0.1569],
         [0.2392, 0.1922, 0.1373,  ..., 0.1020, 0.1137, 0.0784],
         [0.2118, 0.2196, 0.1765,  ..., 0.0941, 0.1333, 0.0824]],

        [[0.4392, 0.4353, 0.4549,  ..., 0.3725, 0.3569, 0.3333],
         [0.4392, 0.4314, 0.4471,  ..., 0.3725, 0.3569, 0.3451],
         [0.4314, 0.4275, 0.4353,  ..., 0.3843, 0.3725, 0.3490],
         ...,
         [0.4863, 0.3922, 0.3451,  ..., 0.3804, 0.2510, 0.3333],
         [0.4549, 0.4000, 0.3333,  ..., 0.3216, 0.3216, 0.2510],
         [0.4196, 0.4118, 0.3490,  ..., 0.3020, 0.3294, 0.2627]],

        [[0.1922, 0.1843, 0.2000,  ..., 0.1412, 0.1412, 0.1294],
         [0.2000, 0.1569, 0.1765,  ..., 0.1216, 0.1255, 0.1333],
         [0.1843, 0.

▍                              128/10112 [00:02<03:00 55.43it/s] [10.16%(13/128)]: 

output tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<SelectBackward>)
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
batch sample: torch.Size([3, 32, 32]) tensor([[[0.3373, 0.3294, 0.3216,  ..., 0.3686, 0.3255, 0.3412],
         [0.3373, 0.3294, 0.3294,  ..., 0.2863, 0.3647, 0.4588],
         [0.3490, 0.3490, 0.3529,  ..., 0.2314, 0.4000, 0.4902],
         ...,
         [0.7647, 0.7647, 0.7804,  ..., 0.7137, 0.6980, 0.7294],
         [0.7804, 0.7725, 0.7843,  ..., 0.7098, 0.7020, 0.7333],
         [0.8235, 0.8000, 0.7922,  ..., 0.7059, 0.6863, 0.7137]],

        [[0.0706, 0

▊                              256/10112 [00:04<02:56 55.99it/s] [10.16%(26/256)]: 

...

█████████████████████████████▌ 9984/10112 [02:59<00:02 44.83it/s] [10.00%(998/9984)]: 

output tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<SelectBackward>)
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
batch sample: torch.Size([3, 32, 32]) tensor([[[0.1255, 0.1725, 0.2431,  ..., 0.1490, 0.1373, 0.1451],
         [0.1059, 0.2039, 0.2314,  ..., 0.1451, 0.1569, 0.1647],
         [0.2157, 0.2431, 0.2314,  ..., 0.1490, 0.1137, 0.1725],
         ...,
         [0.7176, 0.7333, 0.7255,  ..., 0.6235, 0.6275, 0.7059],
         [0.7176, 0.7255, 0.7176,  ..., 0.7569, 0.7569, 0.7373],
         [0.6706, 0.6745, 0.6784,  ..., 0.7020, 0.7059, 0.6980]],

        [[0.1294, 0

██████████████████████████████ 10112/10112 [03:00<00:00 56.17it/s] [10.00%(1000/10000)]: 

output tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<SelectBackward>)
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


10.0

In [22]:
model.eval()
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

# im = images[0]
# # print(im.size())
# im = torch.reshape(im, (1,3,32,32))
# print(images)
# imshow(images)

test_iter = load_iter(data_loader, 'test')

for i, (batch, label) in test_iter:
    print('batch sample:', batch[0], batch[0].size())

    batch, label = batch.to(device), label.to(device)
    batch = 255. * batch
    
    output = model(batch)
    print("batch output:", output)
    
    im = torch.reshape(batch[0], (1,3,32,32))
    output_im = model(im)
    print("single:", output_im)
        
        
im = 255. * im
print('multiplier!!!!!!', im)

# with torch.no_grad():
#     output = model(im)

output = model(im)
print(output)

                               0/10112 [00:00<? ?it/s] 

batch sample: tensor([[[0.6196, 0.6235, 0.6471,  ..., 0.5373, 0.4941, 0.4549],
         [0.5961, 0.5922, 0.6235,  ..., 0.5333, 0.4902, 0.4667],
         [0.5922, 0.5922, 0.6196,  ..., 0.5451, 0.5098, 0.4706],
         ...,
         [0.2667, 0.1647, 0.1216,  ..., 0.1490, 0.0510, 0.1569],
         [0.2392, 0.1922, 0.1373,  ..., 0.1020, 0.1137, 0.0784],
         [0.2118, 0.2196, 0.1765,  ..., 0.0941, 0.1333, 0.0824]],

        [[0.4392, 0.4353, 0.4549,  ..., 0.3725, 0.3569, 0.3333],
         [0.4392, 0.4314, 0.4471,  ..., 0.3725, 0.3569, 0.3451],
         [0.4314, 0.4275, 0.4353,  ..., 0.3843, 0.3725, 0.3490],
         ...,
         [0.4863, 0.3922, 0.3451,  ..., 0.3804, 0.2510, 0.3333],
         [0.4549, 0.4000, 0.3333,  ..., 0.3216, 0.3216, 0.2510],
         [0.4196, 0.4118, 0.3490,  ..., 0.3020, 0.3294, 0.2627]],

        [[0.1922, 0.1843, 0.2000,  ..., 0.1412, 0.1412, 0.1294],
         [0.2000, 0.1569, 0.1765,  ..., 0.1216, 0.1255, 0.1333],
         [0.1843, 0.1294, 0.1412,  ..., 0.13

▍                              128/10112 [00:02<03:17 50.57it/s] 

batch output: tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddmmBackward>)
single: tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)
batch sample: tensor([[[0.3373, 0.3294, 0.3216,  ..., 0.3686, 0.3255, 0.3412],
         [0.3373, 0.3294, 0.3294,  ..., 0.2863, 0.3647, 0.4588],
         [0.3490, 0.3490, 0.3529,  ..., 0.2314, 0.4000, 0.4902],
         ...,
         [0.7647, 0.7647, 0.7804,  ..., 0.7137, 0.6980, 0.7294],
         [0.7804, 0.7725, 0.7843,  ..., 0.7098, 0.7020, 0.7333],
         [0.8235, 0.8000, 0.7922,  ..., 0.7059, 0.6863, 0.7137]],

        [[0.0706, 0.0706, 0.0627,  ..., 0.3412, 0.3216, 0.3333],
         [0.0588, 0.0588, 0.0588,  ..., 0.1961, 0.3255, 0.4039],
         [0.0627, 0.

▊                              256/10112 [00:04<03:13 51.03it/s] 

...

batch output: tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddmmBackward>)
single: tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)
batch sample: tensor([[[0.1176, 0.1373, 0.1373,  ..., 0.1255, 0.1255, 0.1294],
         [0.1255, 0.1412, 0.1412,  ..., 0.3882, 0.2588, 0.1961],
         [0.1333, 0.1608, 0.2471,  ..., 0.9020, 0.7529, 0.5608],
         ...,
         [0.6706, 0.6039, 0.6353,  ..., 0.7216, 0.7216, 0.6510],
         [0.6784, 0.6196, 0.6275,  ..., 0.6667, 0.6706, 0.7020],
         [0.6784, 0.6235, 0.6627,  ..., 0.6784, 0.6824, 0.6863]],

        [[0.0471, 0.0510, 0.0392,  ..., 0.0314, 0.0275, 0.0275],
         [0.0549, 0.0627, 0.0510,  ..., 0.3137, 0.1686, 0.0902],
         [0.0706, 0.

█████████████████████████████▌ 9984/10112 [03:15<00:02 48.19it/s] 

batch output: tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddmmBackward>)
single: tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)
batch sample: tensor([[[0.1255, 0.1725, 0.2431,  ..., 0.1490, 0.1373, 0.1451],
         [0.1059, 0.2039, 0.2314,  ..., 0.1451, 0.1569, 0.1647],
         [0.2157, 0.2431, 0.2314,  ..., 0.1490, 0.1137, 0.1725],
         ...,
         [0.7176, 0.7333, 0.7255,  ..., 0.6235, 0.6275, 0.7059],
         [0.7176, 0.7255, 0.7176,  ..., 0.7569, 0.7569, 0.7373],
         [0.6706, 0.6745, 0.6784,  ..., 0.7020, 0.7059, 0.6980]],

        [[0.1294, 0.2196, 0.2745,  ..., 0.2000, 0.1843, 0.1725],
         [0.1412, 0.2510, 0.2745,  ..., 0.2000, 0.2078, 0.1961],
         [0.2667, 0.

██████████████████████████████ 10112/10112 [03:15<00:00 51.63it/s] 

batch output: tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBack